# Add Wildcard and Null Block
Add special characters to the initial board configuration.

In [17]:
import itertools
import numpy as np
from IPython.display import clear_output
from sys import exit
from math import floor
from random import choice
from scipy.stats import multivariate_normal

## Utility Function


In [2]:
# check if a value is an integer
def is_int(value):
    try: int(value);   return True
    except: return False

In [3]:
class game(object):
    def __init__(self, w, h, n, init_board=None):
        self.player = 0
        self.status = 'active'

        # set up initial board
        if init_board is not None: 
            self.board = init_board
            h, w = init_board.shape
        else: self.board = np.array([[-1]*w]*h)
        print(self.board)
        self.w, self.h, self.n = w, h, n
        self.col_count = [next((h-k-1 for k in range(h)[::-1] if col[k] >-2), h) for col in self.board.T]
        print(self.col_count)
        self.feasible_moves = set(k for k in range(w) if self.col_count[k]<h)

        
    # display the status of the board
    def print_board(self):
        clear_output() # comment out for debugging
        print('Connect-' + str(self.n))
        for k in range(self.h):
            row = ' '.join(str(r) for r in self.board[k,:])
            row = row.replace('-3', '?').replace('-2', '#').replace('-1','.').replace('1','x')
            print('| ' + row + ' |')
        print('-'*(3+2*self.w))
        print('  ' + ' '.join(str(col) for col in range(self.w)))        

    # get user's input
    def col_input(self):
        # ask for and validate input
        while True:
            col = input('Player ' + str(self.player) + ': Enter the column to place your piece ' + 
                        '(' + str(self.player).replace('1','x') + ') ' )
            if col.lower()=='q': print('Thanks for playing!');   exit(0)
            if col=='`': col = 0 # define an alias for 0

            # check for valid input
            if is_int(col): 
                col = int(col)
                if col>=0 and col<self.w and col in self.feasible_moves: break
        return col

    # count connections to the left and right of pos
    def cnct_gt_n(self, vec, pos): 
        count, player = 1, vec[pos]
        
        # check if at least n in length
        if sum(np.logical_or(vec==player, vec==-3)) < self.n: return False

        for v in vec[:pos][::-1]:
            if v==player or v==-3: count += 1
            else: break
        for v in vec[pos+1:]:
            if v==player or v==-3: count += 1
            else: break
        return count >= self.n

    # extract backward diagonal
    def bdiag(self, board, shift=0):
        if shift==0: return [board[k, self.w-k-1] for k in range(min(self.h, self.w))]
        if shift>0: return [board[k, self.w-(k+shift)-1] for k in range(min(self.h, self.w-shift))]
        return [board[k-shift, self.w-k-1] for k in range(min(self.h+shift, self.w))]   

    # check intersecting row, col, diagonals for connect-n
    def connect_n_check(self, row, col):
        forward_diag = np.diag(self.board, col-row)
        backward_diag = self.bdiag(self.board, self.w-1-col-row)
        return (self.cnct_gt_n(self.board[:, col], row) or             # col
                self.cnct_gt_n(self.board[row, :], col) or             # row
                self.cnct_gt_n(forward_diag, min(row, col)) or         # fdiag
                self.cnct_gt_n(backward_diag, min(row, self.w-1-col))) # bdiag

    # update game        
    def update_game(self, col):
        # update col_count
        self.col_count[col] += 1
        
        # skip special pieces
        row = self.h-self.col_count[col]
        while self.board[row, col]<-1: 
            row -= 1 
            self.col_count[col] += 1
            
        # update board, player and feasible moves
        self.board[row, col] = self.player # assign move to player
        self.player = 1*(self.player==0)   # switch players
        if self.col_count[col]==self.h: self.feasible_moves.remove(col)
                    
        # update status
        if self.connect_n_check(row, col): self.status = 'won';   return
        if not self.feasible_moves: self.status = 'draw'
        
    # two-player game
    def play_game(self):
        while True:
            self.print_board()
            col = self.col_input()
            self.update_game(col)
            if self.status!='active':
                self.print_board()
                if self.status=='won': print('Player ' + str(1-self.player) + ' won!')
                else: print('Game is a draw.')
                return            

In [4]:
w, h, n = 4, 4, 4
init_board = np.array([[-1]*w]*h)
init_board[:,2]= -3

g = game(w, h, n, init_board)
g.play_game()

Connect-4
| . . ? . |
| . . ? . |
| . . ? . |
| . . ? . |
-----------
  0 1 2 3
Player 0: Enter the column to place your piece (0) 
Player 0: Enter the column to place your piece (0) 
Player 0: Enter the column to place your piece (0) q
Thanks for playing!


SystemExit: 0

To exit: use 'exit', 'quit', or Ctrl-D.


## Randomize initial board configuration
* choose the board dimensions uniformily at random (from 4-7)
* choose n and in an ad hoc manner
* number of null blocks is a function of the absolute difference in board dimensions
* number of wildcards depends on n and w\*h
* choose location of special pieces uniformily at random

In [10]:
# get game parameters
w, h = choice(range(4, 8)), choice(range(4, 8))
n = round( min(max(w,h)-1, max(3, max(w,h) - 2*np.random.uniform(0, max(w,h)-min(w,h)) ) ) )

# get special piece counts
null_count = round( max(3, max(w,h) - 4*np.random.uniform(0, abs(w-h)) ) )
wild_count = max(0, round(n/2) + int(np.random.normal(0, (w*h)**0.2)))

# calculate locations of special pieces
def lin2sub(ind, w, h): return (int(ind/w), ind % w)
inds = np.random.choice(range(w*h), size = null_count + wild_count, replace=False)
subs = [lin2sub(ind, w, h) for ind in inds]

# make initial board
init_board = np.array([[-1]*w]*h)

# fill in wildcards and NULL blocks
for (i,j) in subs[:wild_count]: init_board[i,j]= -3
for (i,j) in subs[wild_count:]: init_board[i,j]= -2

# print the board
g = game(w, h, n, init_board)
g.print_board()

Connect-6
| . . . . . |
| . ? # . # |
| . . . . . |
| . . . ? . |
| . . . . . |
| . # . . . |
| . ? . . . |
-------------
  0 1 2 3 4


## Other ideas to implement

The positions near the center of the board are "better" in the sense that they can be used in more potential connections.  
* To make the game easier to win choose location of wildcards near the corners/edges of the board.  
* To make the game harder to win choose NULL blocks near the center of the board. 

Apply this heuristic depending on the size (w\*n) of the board.  
* Smaller boards should have wildcards on the edges and NULL blocks in the center.  
* Larger boards can have more wildcards in the center.  

This can be achieved by sampling from a discretized 2-d normal distribution (or 1 minus the distribution) with mean at the center of the board. 

STILL NEEDS BETTER INITIAL BOARD SETUP

In [19]:
# get game parameters
w, h = choice(range(4, 8)), choice(range(4, 8))
n = floor( min(max(w,h)-1, max(3, max(w,h) - 2*np.random.uniform(0, max(w,h)-min(w,h)) ) ) )

# get special piece counts
null_count = round( max(3, max(w,h) - 4*np.random.uniform(0, abs(w-h)) ) )
wild_count = max(0, np.sqrt(null_count) + int(np.random.normal(0, (w*h)**0.2)))

# get special pieces location probabilities
x, y = np.mgrid[0:w, 0:h]
pos = np.empty(x.shape + (2,))
pos[:, :, 0] = x; pos[:, :, 1] = y
rv = multivariate_normal([w/2, floor(h/2-1)], [[1/np.sqrt(w), 0], [0, 1/np.sqrt(w)]])
p_null = rv.pdf(pos).ravel()
p_wild = 1-p_null
p_null = p_null/sum(p_null)
p_wild = p_wild/sum(p_wild)

# calculate locations of special pieces
def lin2sub(ind, w, h): return (int(ind/w), ind % w)

# get location of null blocks
null_inds = np.random.choice(range(w*h), size = null_count, replace=False, p = p_null)
null_subs = [lin2sub(ind, w, h) for ind in null_inds]

# get location of wildcards
keep = [k for k in range(w*h) if k not in null_inds]
p_wild = [p_wild[k] for k in keep]
p_wild = p_wild/sum(p_wild)
wild_inds = np.random.choice(keep, size = wild_count, replace=False, p = p_wild)
wild_subs = [lin2sub(ind, w, h) for ind in wild_inds]

# make initial board
init_board = np.array([[-1]*w]*h)

# fill in wildcards and NULL blocks
for (i,j) in null_subs: init_board[i,j]= -2
for (i,j) in wild_subs: init_board[i,j]= -3

# print the board
g = game(w, h, n, init_board)
g.print_board()

Connect-5
| . ? . . . . |
| . . . . . . |
| . # # . . . |
| . # # # . ? |
| . . # . . . |
| . . . . . . |
---------------
  0 1 2 3 4 5
